In [14]:
import requests
from bs4 import BeautifulSoup
import csv


In [15]:
# Step 1: Fetch the webpage content
url = "https://www.fmscout.com/a-guide-to-player-roles-in-football-manager.html"
response = requests.get(url)
# Check if the request was successful
if response.status_code != 200:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    exit()

In [49]:
# print(response.content)


In [17]:
soup = BeautifulSoup(response.content, 'html.parser')

In [19]:
for h3 in soup.find_all('h3'):
    print(h3.get_text(strip = True))

How do Roles Appear
Goalkeeper Roles
Defending Roles
Midfield Roles
Flank Roles
Central Attack Roles
Where to go From Here?


In [50]:
for h5 in soup.find_all('h5'):
    print(h5.get_text(strip = True))
    description = ""
    p = h5.find_next_sibling('p')
    if p:
        description = p.get_text(strip=True)
    # print(description)

Most Popular


In [51]:
roles = {}
current_role = None

for tag in soup.find_all(['h5', 'blockquote', 'li']):
    if tag.name == 'h5':
        current_role = tag.text
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'blockquote' and current_role:
        roles[current_role]['Definition'] = tag.text.strip()
    elif tag.name == 'li' and current_role:
        roles[current_role]['Roles'].append(tag.text.strip())

# # Print the extracted roles and descriptions
# for role, details in roles.items():
#     print(f"Role: {role}")
#     print(f"Definition: {details['Definition']}")
#     print("Roles:")
#     for r in details['Roles']:
#         print(f" - {r}")
#     print()

In [47]:
roles = {}
current_role = None

for tag in soup.find_all(['h5', 'blockquote', 'li']):
    if tag.name == 'h5':
        current_role = tag.text
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'blockquote' and current_role:
        roles[current_role]['Definition'] = tag.text.strip()
    elif tag.name == 'li' and current_role:
        if '–' in tag.text:
            duty, description = tag.text.split('–', 1)
            roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})
        else:
            roles[current_role]['Roles'].append({'Duty': tag.text.strip(), 'Description': ''})

# # Print the extracted roles and descriptions
# for role, details in roles.items():
#     print(f"Role: {role}")
#     print(f"Definition: {details['Definition']}")
#     print("Roles:")
#     for r in details['Roles']:
#         print(f" - Duty: {r['Duty']}")
#         print(f"   Description: {r['Description']}")
#     print()

In [23]:
import pandas as pd


In [24]:
csv_filename = 'roles.csv'
df = pd.read_csv(csv_filename)

# Print the head and info of the DataFrame
df.head()

,Role,Definition,Duty,Description
0,Goalkeeper,The Goalkeeper role is having your keeper’s ma...,Defend,There is only one role and that is to stop goa...
1,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Defend,The keeper will play a lot more cautiously but...
2,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Support,The keeper will stray just outside their penal...
3,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Attack,A very “brave” keeper who will travel far outs...
4,Wide Centre-Back,The main job of this role is to stop the oppos...,Defend,The player is more like a traditional centre-b...


In [25]:
df[df['Role'] == 'Sweeper Keeper']

,Role,Definition,Duty,Description
1,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Defend,The keeper will play a lot more cautiously but...
2,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Support,The keeper will stray just outside their penal...
3,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Attack,A very “brave” keeper who will travel far outs...


In [26]:
roles = {}
current_role = None

for tag in soup.find_all(['h5', 'blockquote', 'li', 'br']):
    if tag.name == 'h5':
        current_role = tag.text
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'blockquote' and current_role:
        roles[current_role]['Definition'] = tag.text.strip()
    elif tag.name == 'li' and current_role:
        if '–' in tag.text:
            duty, description = tag.text.split('–', 1)
            roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})
        else:
            roles[current_role]['Roles'].append({'Duty': tag.text.strip(), 'Description': ''})
    elif tag.name == 'br' and current_role and tag.next_sibling and '–' in tag.next_sibling:
        duty, description = tag.next_sibling.split('–', 1)
        roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})

# Write the extracted data to a CSV file
csv_filename = 'roles.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Role', 'Definition', 'Duty', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for role, details in roles.items():
        if not details['Roles']:
            writer.writerow({
                'Role': role,
                'Definition': details['Definition'],
                'Duty': '',
                'Description': ''
            })
        else:
            for r in details['Roles']:
                writer.writerow({
                    'Role': role,
                    'Definition': details['Definition'],
                    'Duty': r['Duty'],
                    'Description': r['Description']
                })

print("Data has been written to roles.csv")

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_filename)

Data has been written to roles.csv


In [27]:
df.head()

,Role,Definition,Duty,Description
0,Goalkeeper,The Goalkeeper role is having your keeper’s ma...,Defend,There is only one role and that is to stop goa...
1,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Defend,The keeper will play a lot more cautiously but...
2,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Support,The keeper will stray just outside their penal...
3,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Attack,A very “brave” keeper who will travel far outs...
4,Wide Centre-Back !NEW in FM22!,The main job of this role is to stop the oppos...,Defend - The player is more like a traditional...,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Role         100 non-null    object
 1   Definition   89 non-null     object
 2   Duty         97 non-null     object
 3   Description  85 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [29]:
df = df[~df['Definition'].isnull()]

In [30]:
df[df['Description'].isnull()]

,Role,Definition,Duty,Description
4,Wide Centre-Back !NEW in FM22!,The main job of this role is to stop the oppos...,Defend - The player is more like a traditional...,NaN
5,Wide Centre-Back !NEW in FM22!,The main job of this role is to stop the oppos...,"Support - In the supportive role, the wide cen...",NaN
6,Wide Centre-Back !NEW in FM22!,The main job of this role is to stop the oppos...,"Attack - With an attack duty, the player is wi...",NaN
49,Complete Wing Back,The complete wing back is competent in defensi...,Support - Mostly be a supportive role and be s...,NaN


In [31]:
import json

In [48]:
roles = {}
current_role = None

for tag in soup.find_all(['h5', 'blockquote', 'li', 'br']):
    if tag.name == 'h5':
        # current_role = tag.text
        current_role = tag.text.replace('!NEW in FM22!', '').strip()
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'blockquote' and current_role:
        roles[current_role]['Definition'] = tag.text.strip()
    elif tag.name == 'li' and current_role:
        if '–' in tag.text:
            
            duty, description = tag.text.split('–', 1)
            roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})
        elif '-' in tag.text:
            duty, description = tag.text.split('-', 1)
            roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})
        else:
            roles[current_role]['Roles'].append({'Duty': tag.text.strip(), 'Description': ''})
    elif tag.name == 'br' and current_role and tag.next_sibling and '–' in tag.next_sibling:
        duty, description = tag.next_sibling.split('–', 1)
        roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})

# Write the extracted data to a CSV file
csv_filename = 'roles.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Role', 'Definition', 'Duty', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for role, details in roles.items():
        if not details['Roles']:
            writer.writerow({
                'Role': role,
                'Definition': details['Definition'],
                'Duty': '',
                'Description': ''
            })
        else:
            for r in details['Roles']:
                writer.writerow({
                    'Role': role,
                    'Definition': details['Definition'],
                    'Duty': r['Duty'],
                    'Description': r['Description']
                })

# Write the extracted data to a JSON file
json_filename = 'roles.json'
with open(json_filename, 'w', encoding='utf-8') as jsonfile:
    json.dump(roles, jsonfile, ensure_ascii=False, indent=4)

print("Data has been written to roles.json")

# Print the JSON data
with open(json_filename, 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)
    # print(json.dumps(data, ensure_ascii=False, indent=4))
print("Data has been written to roles.csv")



Data has been written to roles.json
Data has been written to roles.csv


In [33]:
items = list(data.items())

In [34]:
items = items[:-4]

In [35]:
updated_data = dict(items)

In [36]:
updated_data['Goalkeeper']

{'Definition': 'The Goalkeeper role is having your keeper’s main focus on making saves from shots and collecting crosses. Essentially your keeper is the last line of defense and is the “traditional” keeper. Despite being traditional, depending on their attributes they can still play key passes and have excellent distribution.',
 'Roles': [{'Duty': 'Defend',
   'Description': 'There is only one role and that is to stop goals going in. Simple and straightforward, defend the net.'}]}

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 0 to 88
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Role         89 non-null     object
 1   Definition   89 non-null     object
 2   Duty         89 non-null     object
 3   Description  85 non-null     object
dtypes: object(4)
memory usage: 3.5+ KB


In [38]:
df = df[~df['Definition'].isnull()]

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 0 to 88
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Role         89 non-null     object
 1   Definition   89 non-null     object
 2   Duty         89 non-null     object
 3   Description  85 non-null     object
dtypes: object(4)
memory usage: 3.5+ KB


In [40]:
df['Role'].value_counts()

Role
Full Back                         4
Wing Back                         4
Wide Midfielder                   4
Central Midfielder                4
Inverted Wing Back                4
No-Nonsense Centre Back           3
Pressing Forward                  3
Wide Centre-Back !NEW in FM22!    3
Sweeper Keeper                    3
Ball Playing Defender             3
Central Defender                  3
Deep Lying Playmaker              2
Target Man                        2
Complete Forward                  2
Deep Lying Forward                2
Ball Winning Midfielder           2
Defensive Midfielder              2
Libero                            2
Mezzala                           2
Segundo Volante                   2
Winger                            2
Complete Wing Back                2
Defensive Winger                  2
Attacking Midfielder              2
Inverted Winger                   2
Inside Forward                    2
Wide Playmaker                    2
Advanced Playmaker     

In [41]:
df.head()

,Role,Definition,Duty,Description
0,Goalkeeper,The Goalkeeper role is having your keeper’s ma...,Defend,There is only one role and that is to stop goa...
1,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Defend,The keeper will play a lot more cautiously but...
2,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Support,The keeper will stray just outside their penal...
3,Sweeper Keeper,This Goalkeeping role is a mixture between bei...,Attack,A very “brave” keeper who will travel far outs...
4,Wide Centre-Back !NEW in FM22!,The main job of this role is to stop the oppos...,Defend - The player is more like a traditional...,NaN


In [42]:
list(df[df['Role'] == 'Goalkeeper']['Definition'])

['The Goalkeeper role is having your keeper’s main focus on making saves from shots and collecting crosses. Essentially your keeper is the last line of defense and is the “traditional” keeper. Despite being traditional, depending on their attributes they can still play key passes and have excellent distribution.']

In [43]:
url = "https://www.fourfourtwo.com/features/football-manager-2024-every-player-role-in-fm24-explained"
response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')



In [44]:
roles = {}
current_role = None

# Extract roles and descriptions from the website
for tag in soup.find_all(['h2', 'p', 'li']):
    if tag.name == 'h2':
        current_role = tag.text.replace('!NEW in FM22!', '').strip()
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'p' and current_role:
        if 'Definition' not in roles[current_role] or not roles[current_role]['Definition']:
            roles[current_role]['Definition'] = tag.text.strip()
    elif tag.name == 'li' and current_role:
        if '–' in tag.text:
            duty, description = tag.text.split('–', 1)
            roles[current_role]['Roles'].append({'Duty': duty.strip(), 'Description': description.strip()})
        else:
            roles[current_role]['Roles'].append({'Duty': tag.text.strip(), 'Description': ''})
    

In [45]:
# Fetch the HTML content from the URL
url = "https://www.fourfourtwo.com/features/football-manager-2024-every-player-role-in-fm24-explained"
response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

roles = {}
current_role = None

# Extract roles and descriptions from the website
for tag in soup.find_all(['strong', 'p']):
    if tag.name == 'strong':
        current_role = tag.text.replace('!NEW in FM22!', '').strip()
        roles[current_role] = {'Definition': '', 'Roles': []}
    elif tag.name == 'p' and current_role:
        if 'Definition' not in roles[current_role] or not roles[current_role]['Definition']:
            roles[current_role]['Definition'] = tag.text.strip()

In [46]:
roles['Goalkeeper']

{'Definition': "If there's a player role in Football Manager that does what it says on the tin, it's this: the 'traditional', stop-the-ball-from-going-into-the-net goalkeeper who doesn't tend to stray incredibly far from their line (although they can still be good with their feet).",
 'Roles': []}